In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


def get_path_date_list(granularity):
    if granularity == 'daily':
        date_list = [('2018-12', '2018-12-27'), ('2019-06', '2019-06-27'),
                     ('2019-12', '2019-12-27')]
    if granularity == 'weekly':
        date_list = [('2013-12', '2013-12-28'), ('2014-12', '2014-12-27'),
                     ('2015-12', '2015-12-26'), ('2016-12', '2016-12-24'),
                     ('2017-12', '2017-12-23')]
    if granularity == 'monthly':
        date_list = [('2013-12', '2013-12-31'), ('2014-12', '2014-12-31'),
                     ('2015-12', '2015-12-31'), ('2016-12', '2016-12-31'),
                     ('2017-12', '2017-12-31')]
    return date_list


PG_AA_HOSTS = [('internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com', 7432)]
PG_AA_NAME = 'cohort'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select device_id,store_id,date,app_id,kpi,estimate from plproxy.execute_select_nestloop($proxy$
    select device_id,store_id,date,app_id,kpi,estimate
    from mu.app_{}
    where
        date = '{}'
$proxy$) t (device_id SMALLINT,store_id INT,date DATE,app_id BIGINT,kpi SMALLINT, estimate FLOAT) limit 50;"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_plproxy_dump_accuracy(date_list, graularity):
    """
        date_list:
                [(month1,day1), (month1,day2), (month2,day1), (month2,day2)]
        sample:
            [('2015-12', '2015-12-27'), ('2015-12', '2015-12-28'),
            ('2016-12', '2016-12-27'), ('2016-12', '2016-12-28')]
    """
    for date in date_list:
        result = query(aa_dsn, sql.format(graularity, date[1]))
        plproxy_df = spark.createDataFrame(result,
                                           schema=['device_id', 'store_id', 'date', 'app_id', 'kpi', 'estimate'])

        dump_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/" \
                    "granularity={}/month={}/"
        dump_df = spark.read.parquet(dump_path.format(graularity, date[0]))

        join_df_count = dump_df.filter("date='{}'".format(date[1])).join(
            plproxy_df, on=[plproxy_df.device_id == dump_df.device_id, plproxy_df.store_id == dump_df.store_id,
                            plproxy_df.app_id == dump_df.app_id, plproxy_df.kpi == dump_df.kpi,
                            plproxy_df.estimate == dump_df.estimate], how='inner').count()
        if join_df_count != 50:
            print "Accuracy Test Fail!!!! graularity: {}, date: {}".format(graularity, date[1])
        else:
            print "Accuracy Test Pass! graularity: {},date : {}".format(graularity, date[1])


graularity_list = ["daily"]
for graularity in graularity_list:
    check_plproxy_dump_accuracy(get_path_date_list(graularity), graularity)

In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-application-autopipeline")

# reload dependencies from temp
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy